In [6]:
import requests
import os
import json
import pandas as pd

API_ENDPOINT="https://api.osmosis.interbloc.org/osmosis/gamm/v1beta1/pools/1"

UPGRADE_HEIGHT=4707300
HALT_HEIGHT=4713064

## Download all the data 

```
curl \
    -X GET \
    -H "Content-Type: application/json" \
    -H "x-cosmos-block-height: 4707300" \
    "https://api.osmosis.interbloc.org/osmosis/gamm/v1beta1/pools/1"
```

In [7]:
POOLS = [1]
FILE_PATH = 'data/pools/{p}/{h}.json'
STEP = 5

def get_pool_data(pool_id):
    for height in range(UPGRADE_HEIGHT, HALT_HEIGHT):

        if not os.path.exists(FILE_PATH.format(p=pool_id, h=height)):

            headers={
                'Accept': 'application/json',
                'x-cosmos-block-height': str(height),
            }

            response = requests.get(API_ENDPOINT, headers=headers)

            with open(FILE_PATH.format(p=pool_id, h=height), 'w') as f:
                json.dump(response.json(), f)

for pool_id in POOLS:
    get_pool_data(pool_id)